In [2]:
import json
import os
import subprocess
from IPython.display import clear_output

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device is", device)

/home/brandon/Projet_TER/TEST_byMe/env_pompier/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:276: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


device is cuda


# Generate metrics

In [11]:
dic_tarif_zero = {'v_sent': 0,
                  'v_sent_full': 0,
            'v_degraded':0,
            'cancelled':0, #cancel departure
            'function_not_found':0,
            'v1_not_sent_from_1st_station':0,
            'v_not_found_in_last_station':0,
            # 'ff_sent':0,
            # 'ff_skill_mean':0,
            'z1_sent': 0,
            'skill_lvl':0
            }

In [12]:
os.chdir('./Reward_weights')

with open("rw_skill_lvl_r100_cf3.json", "r") as f:
    d = json.load(f)
os.chdir("../")
d

{'v_sent': 0,
 'v_sent_full': 0,
 'v_degraded': 0,
 'cancelled': 0,
 'function_not_found': 0,
 'v1_not_sent_from_1st_station': 0,
 'v_not_found_in_last_station': 0,
 'z1_sent': 0,
 'skill_lvl': 100}

# Agent params

In [ ]:
cmd = ["python3", "-u", "agent_pomo_run.py"]

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

line_count = 0
for line in process.stdout:
    line_count += 1
    if line_count % 100 == 0:
        clear_output(wait=True)

    print(line.strip())

/home/brandon/Projet_TER/TEST_byMe

Bien arrivé dans agent_pomo_run !

Nombre d'agent = 5. on démarre le multi process ...

Bien arrivé dans la fonction run_agent !
La commande de l'agent agent_z1_pomo_sent__r100_cf3_agent_0 est ['python3', 'agent_run.py', '--model_name', 'agent_z1_pomo_sent__r100_cf3_agent_0', '--hyper_params', 'hyper_params.json', '--reward_weights', 'rw_z1_sent_r100_cf3.json', '--dataset', 'df_pc_fake_1y.pkl', '--start', '1', '--end', '53088', '--constraint_factor_veh', '3', '--constraint_factor_ff', '1', '--save_metrics_as', 'agent_pomo_metrics_z1_pomo_sent__r100_cf3_agent_0', '--train']


Bien arrivé dans la fonction run_agent !
La commande de l'agent agent_z1_pomo_sent__r100_cf3_agent_1 est ['python3', 'agent_run.py', '--model_name', 'agent_z1_pomo_sent__r100_cf3_agent_1', '--hyper_params', 'hyper_params.json', '--reward_weights', 'rw_z1_sent_r100_cf3.json', '--dataset', 'df_pc_fake_1y.pkl', '--start', '1', '--end', '53088', '--constraint_factor_veh', '3', '--con

KeyboardInterrupt: 